In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN','NAME','STATUS','SPECIAL_CONSIDERATIONS'. Reduce one additional input variable.
application_df= application_df.drop(['EIN', 'NAME', 'STATUS','SPECIAL_CONSIDERATIONS'],1)
application_df.head()

<ipython-input-2-fcf1cacd74ab>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df= application_df.drop(['EIN', 'NAME', 'STATUS','SPECIAL_CONSIDERATIONS'],1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [3]:
# Bucket APPLICATION_TYPE
application_counts = application_df["APPLICATION_TYPE"].value_counts()

# Determine which values to replace if counts are less than 500
replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app, "Other")

# Bucket CLASSIFICATION
classification_counts = application_df["CLASSIFICATION"].value_counts()

# Determine which values to replace if counts are less than 1800
replace_class = list(classification_counts[classification_counts < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")

# Generate our categorical variable lists
application_cat = list(application_df.dtypes[application_df.dtypes == "object"].index)

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(labels=application_cat, axis=1)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [4]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values.reshape(-1, 1)
X = application_df.drop("IS_SUCCESSFUL", axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [5]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer. Increase the number of hidden layers to 3.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 30

# Number_input_features
nn_inc_epochs = tf.keras.models.Sequential()

# First hidden layer
nn_inc_epochs.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_inc_epochs.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_inc_epochs.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn_inc_epochs.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_inc_epochs.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3280      
                                                                 
 dense_1 (Dense)             (None, 50)                4050      
                                                                 
 dense_2 (Dense)             (None, 30)                1530      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 8,891
Trainable params: 8,891
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Compile the model
nn_inc_epochs.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [7]:
# Train the Model
fit_model_inc_epochs = nn_inc_epochs.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5695 - accuracy: 0.7209
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5538 - accuracy: 0.7300
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5522 - accuracy: 0.7306
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5501 - accuracy: 0.7297
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7305
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5483 - accuracy: 0.7311
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7338
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7325
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5461 - accuracy: 0.7323
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5449 - accura

In [9]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_inc_epochs.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5677 - accuracy: 0.7297 - 223ms/epoch - 832us/step
Loss: 0.5677396059036255, Accuracy: 0.72967928647995


In [ ]:
# Changing activation function from relu to tanh

In [15]:
# Changing Activation Function: try tanh to handle negative inputs in X_train_scaled
nn_tanh = tf.keras.models.Sequential()

# First hidden layer
nn_tanh.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn_tanh.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn_tanh.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn_tanh.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_tanh.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 80)                3280      
                                                                 
 dense_12 (Dense)            (None, 50)                4050      
                                                                 
 dense_13 (Dense)            (None, 30)                1530      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 8,891
Trainable params: 8,891
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn_tanh.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the Model
fit_model_tanh = nn_tanh.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5686 - accuracy: 0.7231
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5560 - accuracy: 0.7286
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5533 - accuracy: 0.7296
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5507 - accuracy: 0.7288
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5491 - accuracy: 0.7291
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7308
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7309
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5453 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5447 - accuracy: 0.7325
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5439 - accura

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_tanh.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5600 - accuracy: 0.7261 - 317ms/epoch - 1ms/step
Loss: 0.5599533915519714, Accuracy: 0.726064145565033


In [29]:
# Export and save model to HDF5 file
nn_tanh.save("AlphabetSoupCharity_Optimization.h5")